In [2]:
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical 
# 학습률
from tensorflow.keras.optimizers import Adam

In [3]:
# npz 파일 로딩
data = np.load('./data/cat_dog.npz')

In [4]:
X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']
y_train = data['y_train']
y_val = data['y_val']
y_test = data['y_test']

In [5]:
X_train.shape, X_val.shape, X_test.shape

((1500, 224, 224, 3), (500, 224, 224, 3), (1000, 224, 224, 3))

### 2 VGG 16 전이학습

In [6]:
from tensorflow.keras.applications import VGG16

In [8]:
vgg16 = VGG16(input_shape=(224,224,3), include_top=False, weights='imagenet')

58892288/58889256 [==============================] - 140s 2us/step


In [9]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### 2.1 모델의 전반부를 학습이 불가능하게 잠그기

In [14]:
for layer in vgg16.layers : 
    print(layer.name)
    if layer.name == 'block5_conv3' :
        layer.trainable = True
    else : 
        layer.trainable = False
          

input_2
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [17]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [18]:
from tensorflow.keras.models import Sequential

In [19]:
# 분류기 부착
transfer_model = Sequential()
transfer_model.add(vgg16)
transfer_model.add(Flatten())
transfer_model.add(Dense(128, activation="relu"))
transfer_model.add(Dense(1,activation="sigmoid"))

In [20]:
transfer_model.compile(loss="binary_crossentropy",
                      optimizer="Adam",
                      metrics=['accuracy'])

In [23]:
transfer_model = transfer_model.fit(X_train,
                       y_train,
                        validation_data = (X_val, y_val),
                        epochs=5)

Train on 1500 samples, validate on 500 samples
Epoch 1/5
1500/1500 [==============================] - 19s 13ms/sample - loss: 2.8220 - accuracy: 0.9073 - val_loss: 0.2513 - val_accuracy: 0.9440
Epoch 2/5
1500/1500 [==============================] - 8s 6ms/sample - loss: 0.0678 - accuracy: 0.9840 - val_loss: 0.2390 - val_accuracy: 0.9700
Epoch 3/5
1500/1500 [==============================] - 9s 6ms/sample - loss: 0.0555 - accuracy: 0.9873 - val_loss: 0.2942 - val_accuracy: 0.9660
Epoch 4/5
1500/1500 [==============================] - 8s 6ms/sample - loss: 0.0636 - accuracy: 0.9893 - val_loss: 0.4754 - val_accuracy: 0.9420
Epoch 5/5
1500/1500 [==============================] - 8s 6ms/sample - loss: 0.0726 - accuracy: 0.9827 - val_loss: 0.1503 - val_accuracy: 0.9700
